# **1. Importing Dependencies**

In [ ]:
!python -m spacy download en_core_web_lg
!pip install langchain-experimental presidio-analyzer presidio-anonymizer spacy Faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install langchain-google-genai
import spacy
from langchain_experimental.data_anonymizer import PresidioAnonymizer
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_core.runnables import RunnableSequence
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 13.9 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

# **2. Model Defining**

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    model_name="moonshotai/kimi-k2-instruct-0905",
    temperature=0.3
)

# **3. Structured-Data**

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

class NeuroUserProfile(BaseModel):
    user_id: str

    # 🧠 Cognitive traits
    neurodivergence: Literal["ADHD", "Dyslexia", "Autism"]

    # ⚡ Energy patterns
    break_interval_minutes: int = Field(ge=5,le=120,description="Break interval in minutes between work sessions")
    fatigue_triggers: Optional[List[str]] = Field(default=None,description="Factors that commonly cause fatigue (e.g. noise, long sessions)")


    # 🗣️ AI behavior preferences
    ai_tone: List[Literal["calm", "Friendly", "strict"]]
    response_verbosity: int = Field( ge=1, le=5, description="Controls response detail level (1=simple and short, 5=detailed)")
    step_granularity: Literal["micro", "normal", "macro"]


# **4. Output Parser**

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
class BreakdownStep(BaseModel):
    step_number: int = Field(description="Step number in the breakdown")
    step_task: str = Field(description="generate one and small single step at a time, dont use two sentence)")
    estimated_time_minutes: int = Field(description="estimated minutes to complete the action")

class NeuroTaskBreakdown(BaseModel):
    task_name: str = Field(description="Name of the task")
    dificulty_level: int = Field(ge=1, le=5, description="Difficulty level on a scale of 1-5")
    breakdown: List[BreakdownStep] = Field(description="High-level phases of the task")

parser = JsonOutputParser(pydantic_object=NeuroTaskBreakdown)

# **5. Prompt Template**

In [ ]:
neuro_prompt = PromptTemplate(
    template="""You are a neurodivergent-friendly productivity assistant specializing in breaking down tasks for individuals with {neurodivergence}.

📋 USER PROFILE:
- User ID: {user_id}
- Neurodivergence: {neurodivergence}
- Break Interval: Every {break_interval_minutes} minutes
- Fatigue Triggers: {fatigue_triggers}
- AI Tone: {ai_tone}
- Response Detail Level: {response_verbosity}/5
- Step Granularity: {step_granularity}

🎯 TASK TO BREAK DOWN:
{task_description}

INSTRUCTIONS:
Based on the user's cognitive profile, create a neurodivergent-friendly task breakdown with micro wins.

IMPORTANT CONSIDERATIONS:

1. **Step Granularity**:
   - micro: 8 -10 steps
   - normal: 6-8 steps
   - macro: 4-6 steps

   - use max 20 words in each step

   Always adjust:
   - number of breakdown steps

   Current setting: {step_granularity}

2. **Neurodivergence-Specific Adaptations**:
   - ADHD: Extra clear transitions
   - Dyslexia: Simple language, use bullet points
   - Autism: Predictable structure, clear expectations

3. **Energy**:
   - Avoid fatigue triggers: {fatigue_triggers}

4. **AI Tone ({ai_tone})**:
   - "calm": Gentle, reassuring language
   - "energetic": Upbeat, exciting language
   - "strict": Direct, no-nonsense

5. **Response Verbosity ({response_verbosity}/5)**:
   - 1-2: Minimal, bullet points only
   - 3: Balanced, brief explanations
   - 4-5: Detailed, comprehensive guidance

{format_instructions}

Generate the task breakdown now:""",
    input_variables=[
        "user_id",
        "neurodivergence",
        "break_interval_minutes",
        "fatigue_triggers",
        "ai_tone",
        "response_verbosity",
        "step_granularity",
        "task_description"
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# **6. PII Masking**

In [ ]:
!pip install presidio-analyzer presidio-anonymizer

In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
import re

analyzer = AnalyzerEngine(supported_languages=["en"])
anonymizer = AnonymizerEngine()

task_description = "i want to do my match assignment"

results = analyzer.analyze(text=task_description, language="en")
masked_result = anonymizer.anonymize(text=task_description, analyzer_results=results)

masked_text = masked_result.text
masked_text = re.sub(r'[\u0900-\u097F]+', '[NAME]', masked_text)

print("Original:", task_description)
print("Masked:", masked_text)

Original: i want to do my match assignment
Masked: i want to do my match assignment


# **6. Chain**

In [ ]:
chain = neuro_prompt | llm | parser

In [ ]:
result = chain.invoke(
    {
        "user_id": "123",
        "neurodivergence": "Autism",
        "break_interval_minutes": "5",
        "fatigue_triggers" : "long paragraphs",
        "ai_tone" : "calm",
        "response_verbosity": "1",
        "step_granularity" : "micro",
        "task_description" : task_description
    }
)

In [ ]:
result

{'task_name': 'Math assignment',
 'dificulty_level': 3,
 'breakdown': [{'step_number': 1,
   'step_task': 'Open notebook to blank page',
   'estimated_time_minutes': 1},
  {'step_number': 2,
   'step_task': 'Write name and date at top',
   'estimated_time_minutes': 1},
  {'step_number': 3,
   'step_task': 'Read first problem only',
   'estimated_time_minutes': 2},
  {'step_number': 4,
   'step_task': 'Copy the problem down',
   'estimated_time_minutes': 2},
  {'step_number': 5,
   'step_task': 'Solve one small part',
   'estimated_time_minutes': 3},
  {'step_number': 6,
   'step_task': 'Check answer with calculator',
   'estimated_time_minutes': 1},
  {'step_number': 7,
   'step_task': 'Move to next problem',
   'estimated_time_minutes': 1},
  {'step_number': 8,
   'step_task': 'Close notebook when done',
   'estimated_time_minutes': 1}]}